# 衝突回避-ライブデモ（TensorRT）
ResNet18モデルをTensorRTモデルに変換したことで高速処理が可能になりました。

このnotebookでは、JetBotが「free（直進する）」か「blocked（旋回する）」かを検出するTensorRTモデルを使用して、JetBotがカクツキを抑えてなめらかに衝突回避できることを確認できます。

## TensorRTモデルを読み込む

``convert_to_trt.ipynb``ノートブックの指示に従って、すでに``best_model_resnet18_trt.pth``が存在している事を前提とします。

TensorRTモデルの初期化を下記コードでおこないます。

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_resnet18_trt.pth'))

TensorRTはGPUでのみ動作可能なため、GPUを利用するようにモデルを設定します。

In [ ]:
device = torch.device('cuda')
model = model_trt.to(device)
model = model_trt.eval()

### 前処理関数を作成する

モデルを読み込みましたが、まだ少し問題があります。  
学習時の入力画像フォーマットと、OpenCVのカメラ画像フォーマットは一致しません。  
これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像(HWC layout)からTensor(CHW layout)に変換します。
2. トレーニング中に使ったのと同じパラメーターを使用して正規化します。カメラから取得した画像データにはRGBの値があり、1ピクセルはRGBをそれぞれint型で[0、255]の範囲で表したものになります。モデルの入力に使う画像データのRGBはfloat型で[0.0、1.0]の範囲になるため、カメラ画像のRGBをそれぞれ255で割る必要があります。
3. カメラ画像をGPUメモリに転送します。
4. 入力画像データをバッチ配列に変更します。学習時に複数の画像を入力に持っているため、予測時にも1枚の画像であっても入力データは配列にする必要があります。

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

# この値はpytorch ImageNetの学習に使われた正規化のパラメータです。
# カメラ画像はこの値で正規化することが望ましいでしょう。
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    # torchvision.transforms.functional.to_tensor(image)はPIL画像をTensor形式に変換します。つまりHWCをCHWに変換します。
    # これをto(device)でGPUメモリに転送とhalf()でfloat16に変換します。
    image = transforms.functional.to_tensor(image).to(device).half()
    # pytorch ImageNetのパラメータと同じ値を使って[0.0-1.0]の範囲に正規化します。
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    # バッチ配列化した画像データを返します。
    return image[None, ...]

すばらしい、これでカメラ画像をニューラルネットワークの入力フォーマットに変換するための、pre-processing関数を定義できました。　

次はカメラを使うので、一度カメラ用のデーモンを再起動しておきます。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

カメラを起動し、画面に表示します。また、`blocked`である確率を表示するスライダーを作成します。

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

モーターを制御するためにrobotインスタンスを生成します

In [ ]:
from jetbot import Robot

robot = Robot()

次は、カメラの画像が更新されるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラ画像をPre-processingにかけてモデル入力データに変換する
2. モデル推論の実行
3. 推論結果が50%以上の確率で`blocked`の場合は、左に曲がります。それ以外の場合は前進します。

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model_trt(x)
    #print(y)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    #print(y)
    
    prob_blocked = float(y.flatten()[0])
    #print(prob_blocked)
    
    blocked_slider.value = prob_blocked
    
    # blockedの確率が50%未満なら直進します。それ以外は左に旋回します
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラと連動して処理する必要があります。

JetBotでは、traitletsライブラリをもちいてCameraクラスを実装することで実現しています。

## JetBotを動かしてみよう
次のコードで``start jetbot``ボタンと``stop jetbot``ボタンを作成します。  
``start jetbot``ボタンを押すとモデルの初期化が実行され、JetBotが動作し始めます。  
``stop jetbot``ボタンを押すとJetBotが停止します。  
最初の1フレームの実行時にメモリの初期化が実行されるので、ディープラーニングではどんなモデルも最初の1フレームの処理はすこし時間がかかります。

In [ ]:
import ipywidgets
import time

model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

def start_model(c):
    update({'new': camera.value})
    camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera
model_start_button.on_click(start_model)
    
def stop_model(c):
    camera.unobserve(update, names='value')
    time.sleep(1)
    robot.stop()
model_stop_button.on_click(stop_model)

model_widget = ipywidgets.VBox([
    ipywidgets.HBox([image, blocked_slider]),
    ipywidgets.HBox([model_start_button, model_stop_button])
])

display(model_widget)

やったね！JetBotが動作している場合、新しいカメラフレームごとに画像変換、モデル推論、JetBot制御処理が実行されています。

あなたがデータセットを作ったコース上にJetBotを置いてください。そして障害物に到達したときの動作を確認する事ができるでしょう。

### 結論

以上がTensorRTのライブデモです。今はJetBotが衝突を回避しながらなめらかに走行しているのではないでしょうか？  
ResNet18モデルをTensorRT化することで、モデル推論が高速化され、JetBotがカクツキを抑えてなめらかに動作することがわかりました。

collision avoidanceが上手く行かない場合、正しく走行できるように失敗しやすい場所でさらにデータを追加してください。  
このようにうまくいかない場所を中心にデータを収集すれば、JetBotはさらによく動作するはずです。

ここまでで、Collision Avoidanceの学習は終了になります。